In [14]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from A_TOOLBOX import *
import re
import pickle
import bootcampviztools as bt

from sklearn.metrics import median_absolute_error
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

import urllib.request

In [15]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

-----------------------------------------------------------------

## Una vez listo el modelo, toca predecir con el dataset de predicción 

Definición de **modelo que está listo**. 

Tras hacer suficientes pruebas, analizar los datos, hacer feature engineering, probar diferentes modelos con diferentes parámetros, es con este con el que observo mejores métricas y menos overfitting. ¡Cuidado con el overfitting aquí! Si vuestro modelo aprende muy bien de estos datos pero hay overfitting cuando le pasemos los datos desconocidos de `test.csv` nos arriesgamos a que digamos, no salga lo esperado.

### 1. Entrena dicho modelo con TODOS tus datos de train, esto es con `train.csv` al completo.


**CON LAS TRANSFORMACIONES QUE LE HAYAS REALIZADO A `X` INCLUÍDAS.**


Véase:
- Estandarización/Normalización
- Eliminación de Outliers
- Eliminación de columnas
- Creación de columnas nuevas
- Gestión de valores nulos
- Y un largo etcétera de técnicas que como Data Scientist hayas considerado las mejores para tu dataset.

In [16]:
lib_cpu = {
    'Intel Atom': 1,          # CPUs de gama baja
    'Intel Celeron': 1,       # CPUs de gama baja
    'Intel Pentium': 2,       # CPUs de gama media-baja
    'Intel Core M': 2,        # CPUs de gama media-baja
    'Intel Core i3': 3,       # CPUs de gama media
    'Intel Core i5': 4,       # CPUs de gama media-alta
    'Intel Core i7': 5,       # CPUs de gama alta
    'Intel Xeon': 6,          # CPUs profesionales
    'AMD E-Series': 1,        # CPUs de gama baja
    'AMD A6-Series': 2,       # CPUs de gama media-baja
    'AMD A8-Series': 3,       # CPUs de gama media
    'AMD A9-Series': 3,       # CPUs de gama media
    'AMD A10-Series': 4,      # CPUs de gama media-alta
    'AMD A12-Series': 4,      # CPUs de gama media-alta
    'AMD FX': 4,              # CPUs de gama media-alta
    'AMD Ryzen 3': 4,         # CPUs de gama media-alta
    'AMD Ryzen 5': 5,         # CPUs de gama alta
    'AMD Ryzen 7': 5,         # CPUs de gama alta
    'AMD Ryzen 9': 6,         # CPUs profesionales
}

In [17]:
lib_gpu = {
    'Intel Graphics': 1,   # Gráficos integrados de gama baja
    'Intel HD': 1,         # Gráficos integrados de gama baja
    'Intel UHD': 1,        # Gráficos integrados de gama baja
    'Intel Iris': 2,       # Gráficos integrados de gama media-alta
    'AMD Radeon R2': 1,    # Gráficos integrados de gama baja
    'AMD Radeon R4': 1,    # Gráficos integrados de gama baja
    'AMD Radeon R5': 1,    # Gráficos integrados de gama baja
    'AMD Radeon R7': 2,    # Gráficos integrados de gama media-alta
    'AMD Radeon RX 550': 3,# GPUs dedicadas de gama baja
    'AMD Radeon RX 560': 3,# GPUs dedicadas de gama baja
    'AMD Radeon RX 580': 4,# GPUs dedicadas de gama media
    'AMD Radeon R9': 4,    # GPUs dedicadas de gama media
    'AMD FirePro': 6,      # GPUs profesionales
    'Nvidia Quadro': 6,    # GPUs profesionales
    'Nvidia GeForce 920M': 3, # GPUs dedicadas de gama baja
    'Nvidia GeForce 930M': 3, # GPUs dedicadas de gama baja
    'Nvidia GeForce 940M': 3, # GPUs dedicadas de gama baja
    'Nvidia GeForce 940MX': 3,# GPUs dedicadas de gama baja
    'Nvidia GeForce 950M': 3, # GPUs dedicadas de gama baja
    'Nvidia GeForce GTX 960M': 4, # GPUs dedicadas de gama media
    'Nvidia GeForce GTX 965M': 4, # GPUs dedicadas de gama media
    'Nvidia GeForce GTX 970M': 4, # GPUs dedicadas de gama media
    'Nvidia GeForce GTX 980M': 5, # GPUs dedicadas de gama alta
    'Nvidia GeForce GTX 1050': 4, # GPUs dedicadas de gama media
    'Nvidia GeForce GTX 1050 Ti': 4, # GPUs dedicadas de gama media
    'Nvidia GeForce GTX 1060': 5, # GPUs dedicadas de gama alta
    'Nvidia GeForce GTX 1070': 5, # GPUs dedicadas de gama alta
    'Nvidia GeForce GTX 1080': 5, # GPUs dedicadas de gama alta
    'Nvidia GeForce MX130': 3, # GPUs dedicadas de gama baja
    'Nvidia GeForce MX150': 3, # GPUs dedicadas de gama baja
    'Intel Iris Plus Graphics': 2, # Gráficos integrados de gama media-alta
    'Intel HD Graphics 620': 1,    # Gráficos integrados de gama baja
    'Intel HD Graphics 5300': 1,   # Gráficos integrados de gama baja
    'Intel HD Graphics 6000': 1,   # Gráficos integrados de gama baja
}

In [18]:
def convertir_peso(peso):
    return float(peso.replace("kg", ""))

def convertir_ram(memo):
    return float(memo.replace("GB", ""))

def valores_a_GB (value):
    if 'TB' in value:
        return int(float(value.replace('TB', '').strip()) * 1000)
    elif 'GB' in value:
        return int(float(value.replace('GB', '').strip()))
    return 0

def tipo_memoria (storage, storage_type):
    if storage_type not in storage:
        return 0
    matches = re.findall(r'(\d+(?:\.\d+)?\s*TB|\d+\s*GB)\s*' + storage_type, storage)
    total_gb = sum(valores_a_GB(match) for match in matches)
    return total_gb

def obtener_gama_cpu(cpu):
    for key in lib_cpu.keys():
        if key in cpu:
            return lib_cpu[key]
    return 0

def obtener_gama_gpu(gpu):
    for key in lib_gpu.keys():
        if key in gpu:
            return lib_gpu[key]
    return 0

patron_resolucion = re.compile(r'(\d+)x(\d+)')

def calcular_pixeles(resolucion):
    match = patron_resolucion.search(resolucion)
    ancho, alto = map(int, match.groups())
    return ancho * alto

caracteristicas = ['IPS', 'HD', 'Touchscreen', '4K', 'Retina', 'Quad HD+']

def tiene_caracteristica(resolucion, caracteristica):
    return 1 if caracteristica.lower() in resolucion.lower() else 0


In [19]:
train['Peso_kg'] = train['Weight'].apply(convertir_peso)
train['RAM_GB'] = train['Ram'].apply(convertir_ram)
train['SSD_GB'] = train['Memory'].apply(lambda x: tipo_memoria(x, 'SSD'))
train['HDD_GB'] = train['Memory'].apply(lambda x: tipo_memoria(x, 'HDD'))
train['Flash_Storage_GB'] = train['Memory'].apply(lambda x: tipo_memoria(x, 'Flash Storage'))
train['Hybrid_GB'] = train['Memory'].apply(lambda x: tipo_memoria(x, 'Hybrid'))
train['Gama_CPU'] = train['Cpu'].apply(obtener_gama_cpu)
train['Gama_GPU'] = train['Gpu'].apply(obtener_gama_gpu)
train['Num_Pixeles'] = train['ScreenResolution'].apply(calcular_pixeles)
for caracteristica in caracteristicas:
    train[caracteristica] = train['ScreenResolution'].apply(lambda x: tiene_caracteristica(x, caracteristica))
train_cat = pd.get_dummies(train, columns=['TypeName','OpSys','Company'], dtype = int)

In [20]:
features = [
 'Peso_kg',
 'RAM_GB',
 'SSD_GB',
 'Gama_CPU',
 'Gama_GPU',
 'Num_Pixeles',
 'IPS',
 'HD',
 'Touchscreen',
 '4K',
 'TypeName_Gaming',
 'TypeName_Notebook',
 'TypeName_Ultrabook',
 'TypeName_Workstation',
 'OpSys_Chrome OS',
 'OpSys_Linux',
 'OpSys_No OS',
 'OpSys_Windows 10',
 'OpSys_Windows 7',
 'Company_Acer',
 'Company_MSI',
 'Company_Razer']

In [21]:
X_train = train_cat[features]
y_train = train_cat['Price_euros']

In [22]:
lgb= LGBMRegressor(max_depth = 10, random_state = 42, verbose = -100)

grid_4 = {
       'num_leaves': [31, 50, 70],
       'learning_rate': [0.1, 0.01, 0.001],
       'n_estimators': [100, 200, 300],
       'max_depth': [5, 10, 15],
       'min_child_samples': [20, 30, 40],
       'subsample': [0.8, 0.9, 1.0],
       'colsample_bytree': [0.8, 0.9, 1.0]
   }

lgb_grid = GridSearchCV(lgb,
                        param_grid= grid_4,
                        cv = 5,
                        scoring = "neg_mean_absolute_percentage_error")

lgb_grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=LGBMRegressor(max_depth=10, random_state=42,
                                     verbose=-100),
             param_grid={'colsample_bytree': [0.8, 0.9, 1.0],
                         'learning_rate': [0.1, 0.01, 0.001],
                         'max_depth': [5, 10, 15],
                         'min_child_samples': [20, 30, 40],
                         'n_estimators': [100, 200, 300],
                         'num_leaves': [31, 50, 70],
                         'subsample': [0.8, 0.9, 1.0]},
             scoring='neg_mean_absolute_percentage_error')

### 2. Carga los datos de `test.csv` para predecir.

In [23]:
test['Peso_kg'] = test['Weight'].apply(convertir_peso)
test['RAM_GB'] = test['Ram'].apply(convertir_ram)
test['SSD_GB'] = test['Memory'].apply(lambda x: tipo_memoria(x, 'SSD'))
test['HDD_GB'] = test['Memory'].apply(lambda x: tipo_memoria(x, 'HDD'))
test['Flash_Storage_GB'] = test['Memory'].apply(lambda x: tipo_memoria(x, 'Flash Storage'))
test['Hybrid_GB'] = test['Memory'].apply(lambda x: tipo_memoria(x, 'Hybrid'))
test['Gama_CPU'] = test['Cpu'].apply(obtener_gama_cpu)
test['Gama_GPU'] = test['Gpu'].apply(obtener_gama_gpu)
test['Num_Pixeles'] = test['ScreenResolution'].apply(calcular_pixeles)
for caracteristica in caracteristicas:
    test[caracteristica] = test['ScreenResolution'].apply(lambda x: tiene_caracteristica(x, caracteristica))
test_cat = pd.get_dummies(test, columns=['TypeName','OpSys','Company'], dtype = int)

**¿Por qué puede dar error?** 

IMPORTANTE: APLICAR A ESTOS DATOS LO MISMO QUE HAYÁIS APLICADO A LOS DATOS DE ENTRENAMIENTO

- SI EL ARRAY CON EL QUE HICISTEIS `.fit()` ERA DE 4 COLUMNAS, PARA `.predict()` DEBEN SER LAS MISMAS
- SI AL ARRAY CON EL QUE HICISTEIS `.fit()` LO NORMALIZASTEIS, PARA `.predict()` DEBÉIS NORMALIZARLO
- TODO IGUAL SALVO BORRAR FILAS, EL NÚMERO DE ROWS SE DEBE MANTENER EN ESTE SET, PUES LA PREDICCIÓN DEBE TENER 391 FILAS, SI O SI

**Entonces, si al cargar los datos de train usé `index_col=0` para que utilizara la primera columna del conjunto de datos como índice, ¿tendré que hacerlo también para el conjunto `test.csv`?**

In [ ]:
# ¿Qué opináis?
# Sí

In [24]:
X_pred = test_cat[features]

### 3. AHORA puedo hacer la predicción que será lo que subirás a Kaggle. 

**¿Qué es lo que subirás a Kaggle?**

In [26]:
predictions = lgb_grid.predict(X_pred)
predictions

array([2230.88855269,  640.93583464,  355.77022088, 1369.99702668,
       1310.84140777, 1717.51727557, 2992.6903815 ,  488.36364654,
       1830.63582928, 1860.09215793, 1624.90633877,  577.43552239,
        380.78106846,  672.39714521,  389.27563636,  938.91169612,
        453.41861267, 2031.43565003, 1559.6401261 , 2110.13962566,
       1299.80306849, 1198.82296381, 1930.11189917, 1080.45292534,
       2057.05759077,  366.84140946, 1063.97432197,  287.04665142,
       1112.46350148,  544.87049649, 1107.64789781, 1227.25885885,
       2336.4449541 ,  417.28887906,  618.26564431, 1654.10801536,
        799.5035675 ,  832.70927651, 2628.10799554,  569.57781514,
        629.03356233, 1122.44436243, 1368.14805484, 1949.06800681,
       1000.60658335, 2081.43421248,  440.46322323, 2968.81515818,
        280.7265906 , 1383.3674145 , 1343.845816  ,  939.14407629,
        810.09958054,  353.64205892,  969.41239829,  805.2246384 ,
       1015.97129583,  282.1608967 , 1555.44583003, 1065.42944

**¡PERO! Para subir a Kaggle la predicción, ésta tendrá que tener una forma específica y no valdrá otra.**

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [34]:
sample = pd.read_csv("../data/sample_submission.csv") # Esta es mi ruta del archivo, la vuestra puede ser otra

In [35]:
sample.head()

,id,Price_euros
0,1014,752.0
1,845,499.0
2,1151,1747.0
3,1265,245.0
4,573,1179.0


In [36]:
sample.shape

(391, 2)

### 4. Mete tus predicciones en un dataframe. 

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [27]:
test_cat['Price_euros'] = predictions

pred_01 = test_cat
submission = pred_01[['id','Price_euros']]

In [28]:
submission.head()

,id,Price_euros
0,181,2230.888553
1,708,640.935835
2,862,355.770221
3,1064,1369.997027
4,702,1310.841408


In [29]:
submission.shape

(391, 2)

### 5. Pásale el CHEQUEATOR para comprobar que efectivamente está listo para subir a Kaggle.

In [32]:
def chequeator(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.id.all() == sample.id.all():
                print("You're ready to submit!")
                submission.to_csv("submission.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://i.kym-cdn.com/photos/images/facebook/000/747/556/27a.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("")


In [37]:
chequeator(submission) # submission es el nombre que le habríamos puesto a nuestro .csv con los valores que me salieron en la predicción

You're ready to submit!


NameError: name 'Image' is not defined

# Kaggle:

#### Aquí encontrarás todo lo que necesitas saber: https://www.kaggle.com/competitions/desafio-de-prediccion-de-precios-de-portatiles